## 🏙️ Multi-Camera Public Surveillance with VideoDB RTStream

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/real_time_streaming/multicam/Multicam_Public_Surveillance.ipynb)

<img src="https://i.imgur.com/Hzamclh.jpg" alt="Smart City Monitoring" width="400"/>

*Transforming urban surveillance through intelligent AI-powered monitoring*


---

### 🎯 The Challenge: Smart City Monitoring

Modern cities face growing challenges in ensuring public safety. From crowded metros to busy intersections, monitoring many locations in real-time is tough.  

Traditional systems rely on humans watching multiple screens — costly and error-prone.  
**What if AI could monitor all feeds, detect incidents instantly, and alert authorities automatically?**


The Goal → Automate monitoring with AI:
- Detect incidents in real time  
- Trigger instant alerts  
- Provide multi-angle evidence

### 🚀 Enter VideoDB RTStream
**VideoDB RTStream** brings AI-powered intelligence to multi-camera systems.  
In this demo, a **7-camera surveillance network** can:  
- Monitor multiple zones at once  
- Analyze pedestrian behavior in real-time  
- Trigger smart alerts for unusual activity  
- Provide synchronized multi-angle evidence

### 📊  Dataset: WILDTRACK

We use the **WILDTRACK dataset** (EPFL Computer Vision Lab):  
- **7 cameras**, overlapping FOVs  
- **HD 1080p @ 60 fps** with precise calibration  
- **Real pedestrian activity** from ETH Zurich  

👉 For this notebook, we optimize to **720p @ 30 fps** for smoother real-time streaming.

### 🎥 What You'll Build

By the end of this notebook, you’ll:  
- Connect & manage **7 synchronized streams**  
- Run **AI-powered scene analysis**  
- Set up **intelligent event detection & alerts**  
- Create **multi-camera evidence assets** for investigations


*This demo shows how AI turns raw surveillance feeds into actionable intelligence.*

---
### 📦 Step 1:  Install Dependencies

First, let's install the VideoDB SDK and additional packages needed for multi-camera processing.



In [ ]:
%pip -q install videodb

---
### 🔗 Step 2: Connect to VideoDB

Next, let’s establish a connection to **VideoDB** so we can manage multi-camera streams seamlessly.


In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("Connected to VideoDB securely!")

Connected to VideoDB securely!


---
### 🎥 Step 3: Configure Multi-Camera Streams

Now, we'll set up our multi-camera surveillance system. For this demonstration, we're monitoring key public spaces across a city with **seven strategically placed cameras** to ensure public safety and operational efficiency.


In [ ]:
# Multi-camera configuration for public surveillance
CAMERA_CONFIG = {
    "setting_name": "🏙️ City Public Square Surveillance",
    "cameras": {
        "cam1": {
            "name": "Plaza Overview" ,
            "rtsp_url": "rtsp://samples.rts.videodb.io:8554/pub-cam1",
            "position": "High Angle - East",
            "description": "Provides a wide-angle overview of the plaza from a high vantage point, monitoring general crowd flow."
        },
        "cam2": {
            "name": "Main Walkway Cam",
            "rtsp_url": "rtsp://samples.rts.videodb.io:8554/pub-cam2",
            "position": "Walkway - Close Up",
            "description": "Focuses on the main pedestrian walkway, capturing close-up details of foot traffic."
        },
        "cam3": {
            "name": "Stairway Junction",
            "rtsp_url": "rtsp://samples.rts.videodb.io:8554/pub-cam3",
            "position": "Stairs - Junction",
            "description": "Monitors the area where the main plaza meets the stairs, a key junction point."
        },
        "cam4": {
            "name": "Central Plaza Cam",
            "rtsp_url": "rtsp://samples.rts.videodb.io:8554/pub-cam4",
            "position": "Center Plaza - Mid Angle",
            "description": "Offers a comprehensive view of the central area of the plaza."
        },
        "cam5": {
            "name": "Building Entrance Cam",
            "rtsp_url": "rtsp://samples.rts.videodb.io:8554/pub-cam5",
            "position": "Building Steps - Eye Level",
            "description": "Monitors the steps leading up to the main building entrance."
        },
        "cam6": {
            "name": "Wide Plaza View",
            "rtsp_url": "rtsp://samples.rts.videodb.io:8554/pub-cam6",
            "position": "Wide Angle - West",
            "description": "Captures a wide shot of the plaza, including the building facade and surrounding areas."
        },
        "cam7": {
            "name": "Ground-Level Cross View",
            "rtsp_url": "rtsp://samples.rts.videodb.io:8554/pub-cam7",
            "position": "Ground Level - Center",
            "description": "Provides a ground-level view across the plaza, tracking movement between different areas."
        }
    }
}

print(f"🏟️  Setting up: {CAMERA_CONFIG['setting_name']}")
print(f"📹 Camera count: {len(CAMERA_CONFIG['cameras'])}")
print("\n📋 Camera Layout:")
for cam_id, cam_info in CAMERA_CONFIG['cameras'].items():
    print(f"  {cam_id.upper()}: {cam_info['name']} ({cam_info['position']})")


🏟️  Setting up: 🏙️ City Public Square Surveillance
📹 Camera count: 7

📋 Camera Layout:
  CAM1: Plaza Overview (High Angle - East)
  CAM2: Main Walkway Cam (Walkway - Close Up)
  CAM3: Stairway Junction (Stairs - Junction)
  CAM4: Central Plaza Cam (Center Plaza - Mid Angle)
  CAM5: Building Entrance Cam (Building Steps - Eye Level)
  CAM6: Wide Plaza View (Wide Angle - West)
  CAM7: Ground-Level Cross View (Ground Level - Center)


#### 🎥 Live Preview of Raw Streams

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import Video, display

def connect_and_export_stream(rtsp_url, cam_name, duration=5, output_file="clip.mp4"):
    if os.path.exists(output_file): os.remove(output_file)
    os.system(
        f'ffmpeg -loglevel error -hide_banner -y -rtsp_transport tcp -i "{rtsp_url}" -t {duration} -c copy "{output_file}"'
    )
    if os.path.exists(output_file):
        return cam_name, Video(output_file, embed=True, width=600)
    else:
        return cam_name, None

CLIP_DURATION = 10 #Set the Duration here for the clip time

print("🎥 Connecting to all cameras...\n")
futures = []
with ThreadPoolExecutor() as executor:
    for cam_id, cam_info in CAMERA_CONFIG["cameras"].items():
        out_file = f"{cam_id}_clip.mp4"
        futures.append(
            executor.submit(connect_and_export_stream, cam_info["rtsp_url"], cam_info["name"], CLIP_DURATION, out_file)
        )

    for future in as_completed(futures):
        cam_name, clip = future.result()
        if clip:
            print(f"✅ {cam_name} ready:")
            display(clip)
        else:
            print(f"❌ Failed to capture {cam_name}")


🎥 Connecting to all cameras...



---
### 🎯 Step 4: Connect All Camera Streams

Now, let’s connect all **four camera streams** to **VideoDB RTStream** and run them in sync.


*Get all streams*

In [ ]:
# Connect all cameras while reusing existing streams if they already exist
connected_streams = {}
print("🔌 Connecting to all camera streams (reusing existing where possible)...\n")

# Pre-fetch existing streams once to minimize API calls
try:
    existing_streams = {getattr(s, "name", ""): s for s in coll.list_rtstreams()}
except Exception as e:
    print(f"⚠️  Could not list existing streams: {e}")
    existing_streams = {}

🔌 Connecting to all camera streams (reusing existing where possible)...



*Using existing streams if available*

In [ ]:
for cam_id, cam_info in CAMERA_CONFIG.items():
    try:
        name = f"Surveillance_{cam_id.upper()}"
        print(f"Connecting {cam_id.upper()}: {name}")
        stream = coll.connect_rtstream(name=name, rtsp_url=cam_info["rtsp_url"])
        connected_streams[cam_id] = {"stream": stream, "status": "connected", "info": cam_info}
        print(f"✅ Connected: {stream.id}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        connected_streams[cam_id] = {"stream": None, "status": "failed", "error": str(e)}

print(f"\n✅ {sum(1 for s in connected_streams.values() if s['status'] == 'connected')} out of {len(CAMERA_CONFIG)} cameras connected.")

---
### 🗂️ Step 5: Set Up Scene Indexing

Now we’ll build **scene indexes** for each camera stream, enabling AI-powered analysis across all viewpoints.


*a. Index Config*

In [ ]:
# Should be generic for indexing

# Scene indexing configuration
SCENE_INDEX_CONFIG = {
    "batch_config": {
        "type": "time",
        "value": 15,  # Analyze every 15 seconds
        "frame_count": 1
    },
    "prompt": """Analyze this public surveillance footage and identify key activities. Describe:
    1. Individuals or groups with notable items (e.g., Suspicious People Gatherings, People with weired lugguage)
    2. Crowd behavior (e.g., people gathering in large groups, sudden dispersal, running).
    3. Vehicle activity (e.g., cars stopping in unusual places, prolonged idling, vans or trucks).
    4. Unusual object detection (e.g., unattended bags, boxes left in public spaces).
    5. General patterns of movement and any deviations from the norm.

    Be specific about the appearance of individuals and the location of events within this camera's view."""
}

*b. Setup Index*

In [ ]:
# 🔍 List existing scene indexes for connected cameras
for cam_id, cam_data in connected_streams.items():
    if cam_data["status"] != "connected":
        print(f"⏭️ {cam_id.upper()}: Stream not connected")
        continue

    try:
        indexes = cam_data["stream"].list_scene_indexes()
        if indexes:
            print(f"📑 {cam_id.upper()} ({cam_data['info']['name']}):")
            for idx in indexes:
                print(f"   • {idx.name} (ID: {idx.rtstream_index_id}, Status: {getattr(idx, 'status', 'unknown')})")
        else:
            print(f"📑 {cam_id.upper()}: No indexes found")
    except Exception as e:
        print(f"⚠️ {cam_id.upper()}: Failed to list indexes ({e})")

📑 CAM1 (Plaza Overview):
   • Public_Square_Surveillance1_CAM1_Index (ID: 75e34af6516a72c8, Status: running)
📑 CAM2 (Main Walkway Cam):
   • Public_Square_Surveillance1_CAM2_Index (ID: d64732ea82c526d5, Status: running)
📑 CAM3 (Stairway Junction):
   • Public_Square_Surveillance1_CAM3_Index (ID: 8276f034438e631b, Status: running)
📑 CAM4 (Central Plaza Cam):
   • Public_Square_Surveillance1_CAM4_Index (ID: d77035b3d68d94df, Status: running)
📑 CAM5: No indexes found
📑 CAM6: No indexes found
📑 CAM7: No indexes found


In [ ]:
# ⚙️ Setup or reuse scene indexes
scene_indexes = {}
print("🔧 Setting up scene indexes...\n")

for cam_id, cam_data in connected_streams.items():
    if cam_data["status"] != "connected":
        continue

    stream = cam_data["stream"]
    name = f"Public_Surveillance_{cam_id.upper()}_Index"

    try:
        # Check if an index with this name exists
        existing = next((idx for idx in stream.list_scene_indexes() if getattr(idx, "name", "") == name), None)

        if existing:
            print(f"📊 {cam_id.upper()}: Using existing index '{name}' ({existing.rtstream_index_id})")
            if getattr(existing, "status", None) not in ("running", "active", "connected"):
                try:
                    existing.start()
                    print(f"   ▶️ Started index: {existing.rtstream_index_id}")
                except Exception as se:
                    print(f"   ⚠️ Could not start index: {se}")
            scene_indexes[cam_id] = {"index": existing, "index_id": existing.rtstream_index_id, "status": "active"}
        else:
            print(f"📊 {cam_id.upper()}: Creating new index '{name}'...")
            new_idx = stream.index_visuals(
                batch_config=SCENE_INDEX_CONFIG["batch_config"],
                prompt=SCENE_INDEX_CONFIG["prompt"],
                name=name,
            )
            scene_indexes[cam_id] = {"index": new_idx, "index_id": new_idx.rtstream_index_id, "status": "active"}
            print(f"   ✅ Index created: {new_idx.rtstream_index_id}")

    except Exception as e:
        print(f"❌ {cam_id.upper()}: Failed to setup index ({e})")
        scene_indexes[cam_id] = {"index": None, "index_id": None, "status": "failed", "error": str(e)}

# Summary
active = sum(1 for idx in scene_indexes.values() if idx["status"] == "active")
print(f"\n✅ {active} out of {len(connected_streams)} camera indexes are active.")
if active == 0:
    print("⚠️ No active indexes. Some cameras might not have valid streams.")

---
## 🚨 Phase 2: Set and Receive Alerts

In this phase, we’ll configure alert rules and handle incoming alerts from the multi-camera streams.

### ⚙️ Step 1: Configure and setup events

*Alert Config & Events*

In [ ]:
# Define surveillance events
EVENTS_CONFIG = [
    {
        "label": "unattended_luggage",
        "prompt": "Detect any luggage, backpacks, or packages left unattended for more than a minute.",
        "description": "Unattended luggage detected"
    },
    {
        "label": "large_crowd_formation",
        "prompt": "Identify scenes where a large group of people (more than 4) gathers quickly in a concentrated area.",
        "description": "Large crowd forming"
    },
    {
        "label": "person_with_trolley",
        "prompt": "Detect any person walking with a trolley bag or rolling suitcase.",
        "description": "Person with trolley bag"
    },
    {
        "label": "woman_in_red_coat",
        "prompt": "Identify any woman wearing a distinct red coat or jacket.",
        "description": "Woman in red coat spotted"
    },
    {
        "label": "suspicious_loitering",
        "prompt": "Detect individuals or small groups loitering in the same spot for an extended period without clear purpose.",
        "description": "Suspicious loitering detected"
    }
]

# 🔍 List existing events in VideoDB
existing_events_by_label = {}
try:
    for evt in conn.list_events():
        lbl = evt.get("label")
        eid = evt.get("event_id")
        if lbl and eid:
            existing_events_by_label[lbl] = eid
    if existing_events_by_label:
        print("📑 Existing events found:")
        for lbl, eid in existing_events_by_label.items():
            print(f"   • {lbl} ({eid})")
    else:
        print("📑 No existing events found.")
except Exception as e:
    print(f"⚠️ Could not list existing events: {e}")


📑 Existing events found:
   • timeout_called (0bb5f68981208827)
   • person_with_trolley (0c9831d3a5d702b3)
   • large_crowd_formation (1a5585203e9374d1)
   • suspicious_loitering (3307388de055afbe)
   • basket scored (49977929774f8cd7)
   • player foul (53409a9e89f66846)
   • basket_scored (6c08568f54b59644)
   • unattended_luggage (87a452a83bc362ad)
   • timeout called (902fdd5571bacf30)
   • woman_in_red_coat (afe2776cf3f4608c)
   • player_foul (cab9b3430d908e31)


In [ ]:
# 🚀 Create or reuse events in VideoDB
created_events = {}
print("\n🎯 Setting up cross-camera event detection...\n")

for cfg in EVENTS_CONFIG:
    label = cfg["label"]

    if label in existing_events_by_label:
        event_id = existing_events_by_label[label]
        created_events[label] = {"event_id": event_id, "config": cfg, "status": "existing"}
        print(f"📎 Using existing event: {label} ({event_id})")
        continue

    try:
        print(f"📝 Creating event: {label}...")
        event_id = conn.create_event(event_prompt=cfg["prompt"], label=label)
        created_events[label] = {"event_id": event_id, "config": cfg, "status": "created"}
        print(f"   ✅ Created: {event_id}")
    except Exception as e:
        print(f"   ❌ Failed: {e}")
        created_events[label] = {"event_id": None, "config": cfg, "status": "failed", "error": str(e)}

# 📊 Summary
created = sum(1 for e in created_events.values() if e["status"] == "created")
existing = sum(1 for e in created_events.values() if e["status"] == "existing")
print(f"\n🎯 Events ready: {existing} existing, {created} created ({existing+created}/{len(EVENTS_CONFIG)})")

if existing + created:
    print("🚀 Event detection system ready! Monitoring for:")
    for lbl, evt in created_events.items():
        if evt["status"] in ("created", "existing"):
            print(f"   • {lbl}: {evt['config']['description']}")
else:
    print("⚠️ No events ready. Check configuration and retry.")



🎯 Setting up cross-camera event detection...

📎 Using existing event: unattended_luggage (87a452a83bc362ad)
📎 Using existing event: large_crowd_formation (1a5585203e9374d1)
📎 Using existing event: person_with_trolley (0c9831d3a5d702b3)
📎 Using existing event: woman_in_red_coat (afe2776cf3f4608c)
📎 Using existing event: suspicious_loitering (3307388de055afbe)

🎯 Events ready: 5 existing, 0 created (5/5)
🚀 Event detection system ready! Monitoring for:
   • unattended_luggage: Unattended luggage detected
   • large_crowd_formation: Large crowd forming
   • person_with_trolley: Person with trolley bag
   • woman_in_red_coat: Woman in red coat spotted
   • suspicious_loitering: Suspicious loitering detected


---
### 🌐 Step 2: Configure Webhook & Callback

*a. install pyngrok*

In [ ]:
!pip install pyngrok
!pip install flask

*b. Expose a Public Webhook URL (ngrok or fallback)*

In [ ]:
import os
import socket
from pyngrok import ngrok
from google.colab import userdata

def choose_port(start=5001, tries=5):
    """Pick an available local port (default: 5001–5005)."""
    for p in range(start, start + tries):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(("0.0.0.0", p))
                return p
            except OSError:
                continue
    return start

try:
    # Authenticate ngrok if token available
    # token = os.getenv("NGROK_AUTHTOKEN", "").strip()
    token = userdata.get('ngrok_auth')
    if token:
        ngrok.set_auth_token(token)

    WEBHOOK_PORT = choose_port()
    tunnel = ngrok.connect(WEBHOOK_PORT)
    PUBLIC_WEBHOOK_URL = f"{tunnel.public_url}/webhook"
    print(f"🌍 Public webhook URL: {PUBLIC_WEBHOOK_URL}")
except Exception as e:
    PUBLIC_WEBHOOK_URL = ""
    print(f"⚠️ ngrok tunnel not started: {e}")
    print("➡️  Set `WEBHOOK_URL` manually if using external service (Zapier, Pipedream, etc.).")

# Unified callback (used when creating alerts)
ALERT_CALLBACK_URL = PUBLIC_WEBHOOK_URL or ""
if ALERT_CALLBACK_URL:
    print(f"✅ Callback URL configured: {ALERT_CALLBACK_URL}")
else:
    print("⚠️ No callback URL configured — alerts won’t trigger notifications.")


🌍 Public webhook URL: https://9904b2f1d7e2.ngrok-free.app/webhook
✅ Callback URL configured: https://9904b2f1d7e2.ngrok-free.app/webhook


*c. Webhook Receiver (Flask server)*

In [ ]:
import time
import threading
from flask import Flask, request, jsonify

# In-memory store for webhook calls
webhook_data = globals().get("webhook_data", [])
webhook_meta = globals().get("webhook_meta", {"started_at": time.time(), "count": 0})

app = Flask(__name__)

@app.route("/webhook", methods=["POST"])
def webhook():
    payload = request.get_json(silent=True) or {}
    webhook_meta["count"] += 1
    webhook_data.append({
        "received_at": time.time(),
        "headers": dict(request.headers),
        "data": payload
    })
    webhook_data[:] = webhook_data[-500:]  # keep last 500 only
    print(f"📩 Webhook #{webhook_meta['count']} received: {payload}")
    return jsonify({"status": "ok"})

def run_webhook():
    app.run(host="0.0.0.0", port=WEBHOOK_PORT, debug=False, use_reloader=False)

# Start/reuse thread safely
webhook_thread = globals().get("webhook_thread")
if not webhook_thread or not webhook_thread.is_alive():
    webhook_thread = threading.Thread(target=run_webhook, daemon=True)
    webhook_thread.start()
    globals()["webhook_thread"] = webhook_thread
    print(f"🚀 Webhook server running at http://localhost:{WEBHOOK_PORT}/webhook")
else:
    print(f"✅ Webhook server already running at http://localhost:{WEBHOOK_PORT}/webhook")


🚀 Webhook server running at http://localhost:5001/webhook
 * Serving Flask app '__main__'
 * Debug mode: off


---
### 🚨 Step 3: Multi-Camera Alert System

We’ll now set up an **intelligent alerting pipeline** that continuously monitors all connected cameras.  
Whenever a basketball event is detected, the system will trigger a **real-time notification** enriched with **multi-angle evidence clips** for better context.

*a. Callback & Setup*

In [ ]:
# 🚨 Step 3A: Prepare callback + alert storage

# Use callback URL from earlier cells
callback_url = globals().get("ALERT_CALLBACK_URL", "")
if not callback_url:
    print("⚠️ No callback URL configured. Alerts will be created but won't send notifications.")

# Container for alerts per camera
created_alerts = {}
print("🚨 Setting up alerts (reuse existing where possible)\n")


🚨 Setting up alerts (reuse existing where possible)



*b. Create or Reuse Alerts*

In [ ]:
        else:
            try:
                aid = idx.create_alert(evt["event_id"], callback_url=callback_url or None)
                status = "active" if callback_url else "created_no_webhook"
                created_alerts[cam_id][label] = {"alert_id": aid, "event_id": evt["event_id"], "status": status}
                msg = f"   ✅ Created alert for '{label}': {aid}" if callback_url else f"   ⚠️ Created alert for '{label}' (no webhook)"
                print(msg)

In [ ]:
# 📊 Alert system summary
num_total = sum(len(alerts) for alerts in created_alerts.values())
num_ready = sum(
    1 for cam_alerts in created_alerts.values()
    for a in cam_alerts.values()
    if a["status"] in ("active", "existing", "created_no_webhook")
)

print(f"\n🎯 Alert System Summary: {num_ready}/{num_total} alerts ready")
if num_ready:
    print(f"📬 Alerts will POST to: {callback_url or '❌ (no webhook set)'}")
else:
    print("⚠️ No alerts ready. Check indexes, events, and callback URL.")



🎯 Alert System Summary: 35/35 alerts ready
📬 Alerts will POST to: https://9904b2f1d7e2.ngrok-free.app/webhook


---
## 📡 Phase 3: Alerts & Data Processing

With alerts now streaming in from all cameras, this phase focuses on **capturing, processing, and analyzing** those incoming events.  
We’ll store webhook data in-memory, extract useful context, and prepare it for downstream workflows like dashboards, notifications, or automated actions.

In [ ]:
len(webhook_data)

6

In [ ]:
webhook_data[-1:]

[{'received_at': 1757523735.8817182,
  'headers': {'Host': '9904b2f1d7e2.ngrok-free.app',
   'User-Agent': 'Python/3.12 aiohttp/3.11.11',
   'Content-Length': '866',
   'Accept': '*/*',
   'Accept-Encoding': 'gzip, deflate',
   'Content-Type': 'application/json',
   'X-Forwarded-For': '34.228.137.199',
   'X-Forwarded-Host': '9904b2f1d7e2.ngrok-free.app',
   'X-Forwarded-Proto': 'https'},
  'data': {'event_id': 'event-0c9831d3a5d702b3',
   'label': 'person_with_trolley',
   'confidence': 0.95,
   'explanation': "The scene analysis explicitly states: 'A female is seen pulling a black, standard-sized rolling suitcase,' which directly matches the alert context for detecting a person with a trolley bag or rolling suitcase.",
   'timestamp': '2025-09-10T17:02:15.811085+00:00',
   'start_time': '2025-09-10T22:31:50.886736+05:30',
   'end_time': '2025-09-10T22:32:05.886736+05:30',
   'stream_url': 'https://videodb-rt-streaming-service-us-east-1.s3.us-east-1.amazonaws.com/manifests/rts-0199349

---
### 🎯 Step 1: Choose the Event

From the list of recent alerts, select a specific **basketball event** you’d like to explore further.  
This choice will be used to extract the event window (with multiple camera angles) for deeper analysis.


In [ ]:
# 📡 View & select recent alerts (last 10)

# Keep only the last 10 webhook events
recent_alerts = webhook_data[:10] if webhook_data else []

if not recent_alerts:
    print("⚠️ No alerts received yet.")
else:
    # Normalize alerts into a compact list
    cleaned_alerts = []
    for item in recent_alerts:
        data = item.get("data", {})
        cleaned_alerts.append({
            "label": data.get("label", "N/A"),
            "confidence": data.get("confidence", "N/A"),
            "explanation": (data.get("explanation") or "")[:100]+"...",
            "timestamp": data.get("timestamp", "N/A"),
            "start_time": data.get("start_time", "N/A"),
            "end_time": data.get("end_time", "N/A"),
            "stream_url": data.get("stream_url", "N/A"),
            "player_url": data.get("player_url", "N/A"),
            "event_id": data.get("event_id", "N/A"),
        })

    # Display to user
    print("\n📋 Recent Alerts (last 10):\n")
    for i, alert in enumerate(cleaned_alerts, 1):
        print(f"{i}. 🎯 {alert['label']} | "
              f"✅ {alert['confidence']} | 📝 {alert['explanation']}")

    # Interactive selection
    try:
        choice = int(input("\n👉 Select an alert (1–{0}): ".format(len(cleaned_alerts))).strip())
        if 1 <= choice <= len(cleaned_alerts):
            selected = cleaned_alerts[choice - 1]
            print(f"\n✅ Selected: {selected['label']} "
                  f"(Confidence: {selected['confidence']})")
        else:
            print("⚠️ Invalid selection. Please choose a valid alert number.")
    except (ValueError, EOFError):
        print("⚠️ No valid input received. Skipping selection.")



📋 Recent Alerts (last 10):

1. 🎯 person_with_trolley | ✅ 0.95 | 📝 The scene analysis explicitly states: 'A female is seen pulling a black, standard-sized rolling suit...
2. 🎯 person_with_trolley | ✅ 0.95 | 📝 A man with a rolling suitcase was explicitly identified in the mid-left section of the plaza, which ...
3. 🎯 suspicious_loitering | ✅ 0.8 | 📝 An 'Isolated Standing Person with Shoulder Bag' is noted as standing 'relatively still and somewhat ...
4. 🎯 person_with_trolley | ✅ 1.0 | 📝 A male individual was observed pulling a black, standard-sized rolling suitcase, which directly matc...
5. 🎯 person_with_trolley | ✅ 1.0 | 📝 A person pulling a black, standard-sized rolling suitcase was identified on the left side of the pav...
6. 🎯 person_with_trolley | ✅ 0.95 | 📝 The scene analysis explicitly mentions 'A man in a dark jacket and light pants is standing still, lo...

👉 Select an alert (1–6): 5

✅ Selected: person_with_trolley (Confidence: 1.0)


---
###  🎥 Step 2: Retrieve Multi-Camera Feeds for the Same Timestamp

Once an event is selected, we’ll fetch the **synchronized video segments** from all connected cameras.  
This ensures you get a **multi-angle replay** of the same moment in time, making analysis more accurate and contextual.

*a. Parse the timeline*

In [ ]:
import re
from datetime import datetime, timezone

# ⚙️ Config: symmetric padding around the alert window
OFFSET_SECONDS = 10

# 🔎 Regex to extract timestamps from HLS stream URLs
_STREAM_RE = re.compile(r"/(\d{16})-(\d{16})\.m3u8")

def parse_stream_times(url: str):
    """Extract start/end timestamps (in seconds) from stream URL."""
    if not url:
        return None, None
    match = _STREAM_RE.search(url)
    if not match:
        return None, None
    return int(match[1]) / 1e6, int(match[2]) / 1e6

def parse_iso_ts(ts: str):
    """Convert ISO timestamp string to epoch seconds."""
    if not ts:
        return None
    dt = datetime.fromisoformat(ts)
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return int(dt.timestamp())

# 📦 Extract payload from selection
if not selected:
    raise RuntimeError("❌ No event data found in webhook payload.")

label       = selected.get("label", "unknown")
confidence  = selected.get("confidence", "N/A")
explanation = selected.get("explanation", "")
stream_url  = selected.get("stream_url", "")
player_url  = selected.get("player_url", "")
event_id    = selected.get("event_id", "")
timestamp   = selected.get("timestamp", "")

# 🕒 Resolve event time window
start_s, end_s = parse_stream_times(stream_url)

if not (start_s and end_s):
    start_s = parse_iso_ts(selected.get("start_time"))
    end_s   = parse_iso_ts(selected.get("end_time"))

if not (start_s and end_s):
    raise RuntimeError("❌ Could not determine alert time window from payload.")

# Apply symmetric offset
start_adj = max(0, int(start_s) - OFFSET_SECONDS)
end_adj   = int(end_s) + OFFSET_SECONDS

print(f"⏱️ Time window: {start_adj} → {end_adj} (with ±{OFFSET_SECONDS}s padding)")


⏱️ Time window: 1757523700 → 1757523736 (with ±10s padding)


*b. Generating all streams*

In [ ]:
# Display alert details
print("\n" + "="*70)
print(f"🚨 PROCESSING ALERT: {label.upper()}")
print("="*70)
print(f"📊 Confidence: {confidence}")
print(f"🆔 Event ID: {event_id}")
print(f"⏰ Detected at: {timestamp}")
# print(f"📝 Explanation: {explanation}")
# print(f"\n🕐 TIME WINDOW:")
# print(f"   Original: {datetime.fromtimestamp(int(start_s), timezone.utc)} → {datetime.fromtimestamp(int(end_s), timezone.utc)} UTC")
# print(f"   With ±{OFFSET_SECONDS}s: {datetime.fromtimestamp(start_adj, timezone.utc)} → {datetime.fromtimestamp(end_adj, timezone.utc)} UTC")
print(f"\n🎥 Original Alert Stream: {player_url or stream_url}")

# Generate synchronized streams for all connected cameras
if not globals().get("connected_streams"):
    raise RuntimeError("connected_streams not found. Run the connection step first.")



🚨 PROCESSING ALERT: PERSON_WITH_TROLLEY
📊 Confidence: 0.95
🆔 Event ID: event-0c9831d3a5d702b3
⏰ Detected at: 2025-09-10T17:02:15.811085+00:00

🎥 Original Alert Stream: https://console.videodb.io/player?url=https://videodb-rt-streaming-service-us-east-1.s3.us-east-1.amazonaws.com/manifests/rts-01993492-ab3b-7203-89e9-4b2b842fb4c5/1757523710000000-1757523726000000.m3u8


In [ ]:
from videodb import play_stream

multi_camera_streams = {}
print("\n📹 MULTI-CAMERA SYNCHRONIZED STREAMS:")
print("-"*70)

for cam_id, cam_data in connected_streams.items():
    if cam_data.get("status") != "connected" or not cam_data.get("stream"):
        print(f"⏭️ {cam_id.upper()} - {cam_data['info']['name']}: Not connected")
        continue

    try:
        url = cam_data["stream"].generate_stream(start_adj, end_adj)
        player = play_stream(url)
        multi_camera_streams[cam_id] = {
            "camera_name": cam_data["info"]["name"],
            "stream_url": url,
            "player_url": player,
        }
        print(f"✅ {cam_id.upper()} - {cam_data['info']['name']}:")
        # print(f"   📺 {player}")
        # print(url)

    except Exception as e:
        print(f"❌ {cam_id.upper()} - {cam_data['info']['name']}: Failed ({e})")

print(f"\n🎯 SUMMARY: Generated {len(multi_camera_streams)} synchronized camera streams")
# print("💡 Use 'multi_camera_streams' dict for multi-view rendering or timeline composition")

camera_feeds = [ feed["player_url"] for feed in multi_camera_streams.values() ]


📹 MULTI-CAMERA SYNCHRONIZED STREAMS:
----------------------------------------------------------------------
✅ CAM1 - Plaza Overview:
✅ CAM2 - Main Walkway Cam:
✅ CAM3 - Stairway Junction:
✅ CAM4 - Central Plaza Cam:
✅ CAM5 - Building Entrance Cam:
✅ CAM6 - Wide Plaza View:
✅ CAM7 - Ground-Level Cross View:

🎯 SUMMARY: Generated 7 synchronized camera streams


In [ ]:
camera_feeds[0]

---
### 🧹 Finally: Clean Up Resources

To properly manage resources, this final step allows you to **disconnect all active camera streams**. This is crucial for preventing orphaned processes and ensuring the system is ready for its next use.


In [ ]:
# Confirm before stopping all streams
active_streams = [s for s in connected_streams.values() if s.get("status") == "connected" and s.get("stream")]
if not active_streams:
    print("✅ All streams are already disconnected.")
else:
    try:
        confirm = input(f"❓ Stop all {len(active_streams)} active streams? (y/n): ").strip().lower()
        if confirm == 'y':
            print("\n🔌 Disconnecting all streams...")
            for cam_data in active_streams:
                try:
                    cam_data["stream"].stop()
                    cam_name = cam_data["info"]["name"]
                    print(f"   ✅ Stopped stream: {cam_name}")
                except Exception as e:
                    print(f"   ❌ Failed to stop stream {cam_name}: {e}")
            print("\n🧹 All streams have been disconnected.")
        else:
            print("\n👍 Streams will remain active.")
    except (ValueError, EOFError):
        print("\n⚠️ No valid input received. Streams will remain active.")

❓ Stop all 7 active streams? (y/n): y

🔌 Disconnecting all streams...
   ✅ Stopped stream: Plaza Overview
   ✅ Stopped stream: Main Walkway Cam
   ✅ Stopped stream: Stairway Junction
   ✅ Stopped stream: Central Plaza Cam
   ✅ Stopped stream: Building Entrance Cam
   ✅ Stopped stream: Wide Plaza View
   ✅ Stopped stream: Ground-Level Cross View

🧹 All streams have been disconnected.
